In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
# https://huggingface.co/TechxGenus/Meta-Llama-3-70B-Instruct-GPTQ
LLAMA3_BASE_URL = 'https://corellm.wb.ru/llama3/v1'
LLAMA3_MODEL_NAME = 'Meta-Llama-3-70B-Instruct-GPTQ'


DEEPSEEK_BASE_URL = 'https://corellm.wb.ru/deepseek/v1'
DEEPSEEK_MODEL_NAME = 'DeepSeek-R1'


QWEN25_32B_BASE_URL = 'https://corellm.wb.ru/qwen25-coder-14b-instruct/v1/'
QWEN25_32B_MODEL_NAME = 'Qwen25-Coder-14B-Instruct'

In [13]:
import os

RWB_CORELLM_TOKEN = os.getenv("RWB_CORELLM_TOKEN")

In [24]:
SOURCE_ARTICLE_NAME = 'Exploring ChatGPT and its impact on society MA Haque, S Li'

In [26]:
import pandas as pd

df = pd.read_csv('pdf_extracted_text.csv')

,result_id,pdf_content,content_length,has_error
0,VdBeSlL9_TMJ,**What do Learners See in ChatGPT?**\n\n\n**Ch...,330705,False
1,tEdhwBtai2UJ,Volume 6 Issue 4 Year 2025 Pages 755-765\nISSN...,54918,False
2,JYbhWHRjZR8J,"# **Knowledge Building, Multimodal Digital Sto...",495882,False
3,vvWK674pjjMJ,Digital Policy Hub — Working Paper\n\n# Digita...,38979,False
4,X0Vwc_5HcwcJ,# **From Boundary Object to Boundary Vision:**...,516407,False
5,Vh3K8GLTg9sJ,Zeitschrift für Arbeitswissenschaft\n\n[https:...,59655,False
6,BBM06Azg7dsJ,## **OPEN**\n\n\n\n[www.nature.com/scientificr...,84634,False
7,8GGORR0MaSsJ,"OPEN ACCESS\n\n\nEDITED BY\n\nMeishan Zhang,\n...",120205,False
8,upmOB6l0mqcJ,Postdigital Science and Education\nhttps://doi...,87427,False
9,L0bsiVYsSXAJ,"OPEN ACCESS\n\n\nEDITED BY\n\nSirje Virkus,\nT...",135047,False


In [27]:
import os
from openai import OpenAI


base_url = DEEPSEEK_BASE_URL
model_name = DEEPSEEK_MODEL_NAME

client = OpenAI(
    api_key=RWB_CORELLM_TOKEN,
    base_url=base_url,
)



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"You are a helpful assistant. User will give you article. Find article context which references to '{SOURCE_ARTICLE_NAME}'. Write only text from article"
        },
        {
            "role": "user",
            "content": "",
        }
    ],
    model=model_name,
    stream=True
)

for chunk in chat_completion:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")


"Exploring ChatGPT and its impact on society" by Haque and Li (2023) critically examines the societal implications of ChatGPT, addressing ethical concerns like misinformation, bias, job displacement, and privacy. The study emphasizes the need for transparent development, human oversight, and regulatory frameworks to mitigate risks while harnessing ChatGPT's potential in education, healthcare, and creative fields. It calls for interdisciplinary collaboration to align AI advancements with societal well-being.